In [1]:
import torch
import models
import numpy as np
import mcubes
import matplotlib.pyplot as plt

In [2]:
model = torch.load("../monkj.model")

In [3]:
weights = [v.detach().numpy() for k, v in model.named_parameters() if "weight" in k]
biases = [v.detach().numpy() for k, v in model.named_parameters() if "bias" in k]

In [4]:
for w, b in zip(weights, biases):
    print(w.shape, b.shape)

(16, 32) (16,)
(16, 16) (16,)
(1, 16) (1,)


In [5]:
fourier_layer = model.layers[0]
assert isinstance(fourier_layer, models.FourierFeatures), "Invalid model used."
fourier_mapping = fourier_layer.fourier_mappings.detach().numpy()
fourier_alpha = fourier_layer.alpha.detach().numpy()
print(fourier_mapping.shape, fourier_alpha)

(3, 16) 0.46788183


In [6]:
def relu(x):
    return (x > 0) * x

In [7]:
def custom_forward(weights, biases, fourier_mapping, fourier_alpha, z):
    x = z.copy()
    x = np.matmul(2 * np.pi * x, fourier_mapping * fourier_alpha)
    x = np.concatenate([np.sin(x), np.cos(x)], -1).T
    print(x.shape)
    for w, b in zip(weights[:-1], biases[:-1]):
        x = np.dot(w, x) + b.reshape((-1, 1))
        x = relu(x)
    x = np.dot(weights[-1], x) + biases[-1].reshape((-1, 1))
    return x

In [8]:
N = 100
x = np.random.random((N, 3)).astype('float32')
y_custom = custom_forward(weights, biases, fourier_mapping, fourier_alpha, x).reshape(-1)
y = model(torch.tensor(x)).detach().numpy().reshape(-1)
print(y_custom.shape, y.shape)

(32, 100)
(100,) (100,)


In [9]:
print(y_custom[:10])

[-0.10191108  0.7367834   0.36547217  0.4501231   0.20148337  0.730198
  0.45255038  0.25594926  0.08322731  0.13753077]


In [10]:
print(y[:10])

[-0.1019111   0.7367834   0.36547205  0.4501231   0.2014834   0.730198
  0.45255038  0.2559492   0.08322728  0.13753083]


In [153]:
with open("monkj_weights.txt", "w+") as f:
    # fourier vectors (3x16)
    fourier = fourier_mapping * fourier_alpha
    fourier = fourier.reshape(-1)
    f.write(f"{','.join([str(i) for i in fourier])}\n")

    # input -> first layer (16x32) + bias (16x1)
    w0, b0 = weights[0].reshape(-1), biases[0].reshape(-1)
    l0 = np.hstack([w0, b0])
    f.write(f"{','.join([str(i) for i in l0])}\n")

    # first layer -> second layer (16x16) + bias (16x1)
    w1, b1 = weights[1].reshape(-1), biases[1].reshape(-1)
    l1 = np.hstack([w1, b1])
    f.write(f"{','.join([str(i) for i in l1])}\n")

    # second layer -> output (16x1) + bias (1x1)
    w2, b2 = weights[2].reshape(-1), biases[2].reshape(-1)
    l2 = np.hstack([w2, b2])
    f.write(f"{','.join([str(i) for i in l2])}\n")

In [11]:
print(fourier_mapping * fourier_alpha)

[[-0.02349853  0.39764506 -0.5260776  -0.20327765 -0.10694765 -0.07671295
  -0.84258777 -0.5155184   1.0241638   0.24054255  0.37238994 -0.21568038
   0.9938353   0.50075865  0.21242084 -0.03669096]
 [ 0.67350215  0.21080692 -0.8645252   0.17587878 -0.47704843  0.11880753
  -0.25094116  0.13399483  0.51107264 -0.20621534  0.29547423  0.3276528
   0.3154978   0.26080543  0.22184876  0.6498949 ]
 [ 0.34839368 -0.1578471  -1.1771588   0.12724102  0.1503143   0.38558716
  -0.0495448  -0.05548007 -0.40300295 -0.33807364 -0.1641675  -0.33305362
  -0.01392652  0.14899871  0.3113187   0.0315331 ]]
